# Wan 2.1 Video Generation on Kaggle

This notebook generates videos from Mahabharata stories using Wan 2.1 model.

**Two Modes Available:**
1. **JSON Input Mode**: Generate videos from existing `video_prompts_*.json` files
2. **Vector DB Mode**: Fetch from vector database → Generate ~5 scenes → Generate videos

## Setup Instructions

1. **Add GitHub as Data Source**:
   - Click "Add Data" → "GitHub"
   - Enter your repository URL

2. **Configure Notebook Settings** (in sidebar):
   - **Accelerator**: GPU T4 x2
   - **Language**: Python 3
   - **Internet**: On
   - **Persistence**: Files Only (optional)

3. **Set API Key** (for Vector DB mode):
   - Go to Settings → Secrets → Add Secret
   - Name: `GEMINI_API_KEY`
   - Value: Your Gemini API key

4. **Upload JSON File** (for JSON Input mode):
   - Upload `video_prompts_*.json` to `/kaggle/input/`


## 1. Setup and Installation


In [ ]:
# Clone repository (if not already added as data source)
import os
import sys
from pathlib import Path

# If repository is added as data source, it will be in /kaggle/input/
# Otherwise, clone it here
repo_path = Path("/kaggle/working/repo")
if not repo_path.exists():
    # Update with your repository URL
    # !git clone <YOUR_REPO_URL> /kaggle/working/repo
    repo_path = Path("/kaggle/working/repo")
else:
    # Find repo in input directory
    input_dir = Path("/kaggle/input")
    for item in input_dir.iterdir():
        if item.is_dir() and (item / "kaggle").exists():
            repo_path = item
            break

print(f"Repository path: {repo_path}")

# Add kaggle directory to path
kaggle_dir = repo_path / "kaggle"
if not kaggle_dir.exists():
    # Try alternative location
    kaggle_dir = Path("/kaggle/input") / "kaggle"
    if not kaggle_dir.exists():
        kaggle_dir = Path("/kaggle/working") / "kaggle"

sys.path.insert(0, str(kaggle_dir.parent))
sys.path.insert(0, str(kaggle_dir))
print(f"Kaggle directory: {kaggle_dir}")


In [ ]:
# Install dependencies
!pip install -q diffusers>=0.21.0 transformers>=4.30.0 accelerate>=0.20.0
!pip install -q imageio>=2.31.0 imageio-ffmpeg>=0.4.8 sentencepiece
!pip install -q chromadb>=0.4.15 sentence-transformers>=2.2.0
!pip install -q google-generativeai>=0.3.0 pydantic>=2.0.0 pyyaml>=6.0.1 numpy>=1.24.0

print("✓ Dependencies installed")


In [ ]:
# Setup Hugging Face cache in /kaggle/tmp/
import os
from pathlib import Path

cache_dir = Path("/kaggle/tmp/huggingface")
cache_dir.mkdir(parents=True, exist_ok=True)

os.environ['HF_HOME'] = str(cache_dir)
os.environ['TRANSFORMERS_CACHE'] = str(cache_dir)
os.environ['HF_DATASETS_CACHE'] = str(cache_dir)

print(f"✓ Hugging Face cache set to: {cache_dir}")


## 2. Configuration

**Select your mode:**
- Set `MODE = "json"` for JSON Input mode
- Set `MODE = "vector_db"` for Vector DB → Story → Video mode


In [ ]:
# Configuration
MODE = "json"  # Change to "vector_db" for story generation mode

# For JSON mode:
JSON_FILE = "/kaggle/input/video_prompts_20251226_214744.json"  # Update with your file path

# For Vector DB mode:
QUERY = None  # Optional: "Arjuna" or "Kurukshetra" for targeted retrieval, None for random

print(f"Mode: {MODE}")
if MODE == "json":
    print(f"JSON File: {JSON_FILE}")
else:
    print(f"Query: {QUERY if QUERY else 'Random'}")


## 3. Mode 1: JSON Input → Video Generation


In [ ]:
if MODE == "json":
    import sys
    from pathlib import Path
    
    # Import generator
    sys.path.insert(0, str(kaggle_dir))
    from wan21_generator import Wan21KaggleGenerator
    
    # Initialize generator
    config_path = kaggle_dir / "kaggle_config.yaml"
    generator = Wan21KaggleGenerator(config_path=config_path)
    
    # Generate videos
    print("\n" + "="*60)
    print("GENERATING VIDEOS FROM JSON")
    print("="*60)
    
    video_paths = generator.generate_from_json(JSON_FILE)
    
    print("\n" + "="*60)
    print("GENERATION COMPLETE")
    print("="*60)
    print(f"\nGenerated {len(video_paths)} videos:")
    for path in video_paths:
        print(f"  - {path}")


## 4. Mode 2: Vector DB → Story Generation → Video Generation


In [ ]:
if MODE == "vector_db":
    import sys
    import json
    from pathlib import Path
    from datetime import datetime
    
    # Import modules
    sys.path.insert(0, str(kaggle_dir))
    from story_pipeline import (
        KaggleVectorStore,
        KaggleRetriever,
        KaggleStoryGenerator,
        KaggleNarrationGenerator
    )
    from wan21_generator import Wan21KaggleGenerator
    
    config_path = kaggle_dir / "kaggle_config.yaml"
    
    print("\n" + "="*60)
    print("STEP 1: LOADING VECTOR DATABASE")
    print("="*60)
    
    # Load vector store
    vector_store = KaggleVectorStore(config_path=config_path)
    print(f"Vector DB loaded: {vector_store.count()} passages")
    
    if vector_store.count() == 0:
        print("\n⚠ WARNING: Vector database is empty!")
        print("You need to either:")
        print("  1. Include pre-built vector_db/ directory in your GitHub repo")
        print("  2. Or recreate it from PDF (see README for instructions)")
        raise ValueError("Vector database is empty")
    
    print("\n" + "="*60)
    print("STEP 2: RETRIEVING PASSAGES")
    print("="*60)
    
    # Initialize retriever
    retriever = KaggleRetriever(vector_store=vector_store, config_path=config_path)
    
    # Retrieve seed passage
    seed_passages = retriever.retrieve_diverse(query=QUERY, n_results=1)
    if not seed_passages:
        raise ValueError("Failed to retrieve seed passage")
    
    seed_passage = seed_passages[0]
    print(f"\nSeed Passage ID: {seed_passage['id']}")
    print(f"Text preview: {seed_passage['text'][:200]}...")
    
    # Retrieve context passages
    context_passages = retriever.retrieve_context(seed_passage, n_context=3)
    print(f"\nRetrieved {len(context_passages)} context passages")
    
    print("\n" + "="*60)
    print("STEP 3: GENERATING STORY")
    print("="*60)
    
    # Generate story
    story_generator = KaggleStoryGenerator(config_path=config_path)
    
    story_data = story_generator.generate_story(
        seed_passage=seed_passage['text'],
        context_passages=[p['text'] for p in context_passages]
    )
    
    print(f"\n✓ Generated story with {len(story_data.get('story_sequence', []))} scenes")
    
    # Generate narration (placeholder)
    narration_generator = KaggleNarrationGenerator(config_path=config_path)
    narration_data = narration_generator.generate_narration(story_data)
    
    # Save generated scripts
    output_dir = Path("/kaggle/working/output")
    output_dir.mkdir(parents=True, exist_ok=True)
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Convert to video_prompts format
    video_prompts = []
    for scene in story_data.get('story_sequence', []):
        clip_prompt = scene.get('clip_prompt', {})
        video_prompts.append({
            "scene_number": scene.get('scene_number', len(video_prompts) + 1),
            "title": scene.get('title', f"Scene {scene.get('scene_number', len(video_prompts) + 1)}"),
            "video_prompt": clip_prompt.get('visual_description', ''),
            "camera_angles": clip_prompt.get('camera_angles', ''),
            "key_objects": clip_prompt.get('key_objects', ''),
            "ambient_audio": clip_prompt.get('ambient_audio', ''),
            "duration_seconds": 5
        })
    
    # Save video prompts
    prompts_file = output_dir / f"video_prompts_{timestamp}.json"
    with open(prompts_file, 'w', encoding='utf-8') as f:
        json.dump(video_prompts, f, indent=2, ensure_ascii=False)
    print(f"\n✓ Saved video prompts: {prompts_file}")
    
    # Save narration
    narration_file = output_dir / f"narration_{timestamp}.json"
    with open(narration_file, 'w', encoding='utf-8') as f:
        json.dump(narration_data, f, indent=2, ensure_ascii=False)
    print(f"✓ Saved narration: {narration_file}")
    
    print("\n" + "="*60)
    print("STEP 4: GENERATING VIDEOS")
    print("="*60)
    
    # Initialize video generator
    video_generator = Wan21KaggleGenerator(config_path=config_path)
    
    # Generate videos from scenes
    video_paths = video_generator.generate_from_scenes(video_prompts)
    
    print("\n" + "="*60)
    print("GENERATION COMPLETE")
    print("="*60)
    print(f"\nGenerated {len(video_paths)} videos:")
    for path in video_paths:
        print(f"  - {path}")


In [ ]:
# Zip output folder for easy download
import shutil

output_dir = Path("/kaggle/working/output")
zip_path = Path("/kaggle/working/videos_output.zip")

if output_dir.exists() and any(output_dir.iterdir()):
    shutil.make_archive(str(zip_path).replace('.zip', ''), 'zip', output_dir)
    print(f"✓ Created zip file: {zip_path}")
    print(f"  Size: {zip_path.stat().st_size / (1024*1024):.2f} MB")
else:
    print("No output files to zip")
